In [ ]:
# Python 3.X
import numpy as np
from pandas import Series, DataFrame
from pandas.tseries.offsets import MonthEnd
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ttest_1samp
# np.set_printoptions(precision=|3)
import datetime
plt.rc('figure', figsize=(10, 8))
from numpy.linalg import inv
from numpy import dot
import re

# FINM 8006 Assignment

## Mean Variance Optimization

Data of monthly returns is given in <font color=red>asgn1.csv</font>, where date is the end of the return months, RET is the montly return, PERMNO is the stock code, rf is the risk free rate.

### Q1.1 Mean-Variance Without Risk Free Assets

If your desired rate of return is 3% in the month, find the **optimal weights on the stocks, and the expected mean and standard deviation of your portfolio.** 

In [ ]:
def load_mv_data():
    stocks=DataFrame([{'Apple':14593, 'IBM':12490, 'Microsoft':10107, 'Nike':57665, 'Disney':26403}], index=['PERMNO']).T
    stocks.index.name = 'stocks'
    stocks = stocks.reset_index().sort_values(['PERMNO'])
    msf = pd.read_csv('asgn1.csv')
    msf['date']=pd.to_datetime(msf['date'])
    msf = msf.merge(stocks, on='PERMNO')
    msf = msf.sort_values(['PERMNO', 'date'])
    return msf, stocks
msf, stocks = load_mv_data()
msf.head(3)

,date,PERMNO,RET,rf,stocks
0,1994-01-31,10107,0.055814,0.0025,Microsoft
1,1994-02-28,10107,-0.030837,0.0021,Microsoft
2,1994-03-31,10107,0.027273,0.0027,Microsoft


**1. find efficient frontier**

In [ ]:
# mean and variance
rbar = msf.groupby('PERMNO')['RET'].mean()
cov = msf.reset_index().set_index(['PERMNO', 'date']).unstack('PERMNO')
cov0 =cov['RET'].cov()
cov0

PERMNO,10107,12490,14593,26403,57665
PERMNO,,,,,
10107,0.008903,0.003542,0.004155,0.001984,0.001483
12490,0.003542,0.006065,0.003676,0.002185,0.001023
14593,0.004155,0.003676,0.016855,0.002102,0.000695
26403,0.001984,0.002185,0.002102,0.005439,0.001800
57665,0.001483,0.001023,0.000695,0.001800,0.007197


In [ ]:
# convert from df to matrix
rbar = np.matrix(rbar.values).T
cov0 = np.matrix(cov0.values)
print("1. E(R) of each stocks:", rbar)
print("2. Covariance:\n", cov0)

1. E(R) of each stocks: [[0.01735788]
 [0.01283327]
 [0.0273573 ]
 [0.01158895]
 [0.0193265 ]]
2. Covariance:
 [[0.00890329 0.00354242 0.00415468 0.00198438 0.00148256]
 [0.00354242 0.00606462 0.00367599 0.00218514 0.00102323]
 [0.00415468 0.00367599 0.01685516 0.00210236 0.00069487]
 [0.00198438 0.00218514 0.00210236 0.00543892 0.00180024]
 [0.00148256 0.00102323 0.00069487 0.00180024 0.00719687]]


$\bar{R}= E(R)$

$A = \bar{R}′Σ^{−1}\bar{R}′$; $B = R¯′Σ^{−1}ι$;  $C = ι′Σ^{−1}ι$ => $\lambda_1=\frac{C\mu-B}{AC-B^2}$; $\lambda_2=\frac{A-B\mu}{AC-B^2}$; $\omega=λ_1\sum^{-1}\bar{R}+λ_2*\sum^{−1}$
=> $\hat{σ_p^2}=\frac{A-2B_μ+C_μ^2}{AC-B^2}$

In [ ]:
# mean-variance without rf, lecture 4 
def meanvar(mu, rbar=rbar, cov=cov0): # mu: expected return
    ones = np.ones(rbar.shape) # array([1., 1., 1., 1., 1.])
    A = rbar.T.dot(inv(cov)).dot(rbar)
    B = rbar.T.dot(inv(cov)).dot(ones)
    C = ones.T.dot(inv(cov)).dot(ones)
    omega = (C*mu-B)/(A*C-B**2)*dot(inv(cov), rbar) + (A-B*mu)/(A*C-B**2)*dot(inv(cov), ones) #𝜔
    varhut = np.sqrt(omega.transpose().dot(cov).dot(omega))#\hat{sigma}
    return omega, varhut

meanvar(0.03) 

ValueError: shapes (1,1) and (5,1) not aligned: 1 (dim 1) != 5 (dim 0)

In [ ]:
# weight
PERMNO=list(cov0)
w1 = pd.DataFrame(musigma[0],index=PERMNO,columns=["weight of each stock"])
print('Weight Vector of required rate of return=0.03:')
w1

In [ ]:
# standard deviation
sd = musigma[1]
print('Portfolio return standard deviation is: {:.4f}'.format(sd))

In [ ]:
mus = np.linspace(0.001, 0.06, 100)
sigmas =[]
for m in mus:
 sm = meanvar(m)[1]
 sigmas.append(sm)
zipped =zip(mus, sigmas)

2. optimal risky portfolio (Maximize sharpe ratio）

### Q1.2
Plot the mean-variance efficeint curve of these assets. 

4. 画出CAL

## Asset Pricing in Chinese Markets

Many have commented that the Chinese stock market is very different from the more mature markets. In this assignment, you will explore some aspects of the asset pricing in Chinese markets.

#### Loading data

You are given three data sets from year 2000: 'csmar_be.csv' has the December book value (BE) of firms (STKCD). 'csmarstk0016.csv' has monthly market value (ME) and return (ret) information of individual stocks. Data is cleaned to exclude micro stoks and penny stocks. 'csmarmktretrf.csv' has market and risk free returns. Market return here is proxied by returns on the SSE. 

* ME: market value of equity
* STKCD: stock code
* MARKETTYPE: market type, 1 is Shanghai, 4 is Shengzhen
* BE: book value of equity
* ret: stock return
* mktret: market return
* rf: risk free rate

In [ ]:
def load_stock_returns():
    # markettype 1 Shanghai A, 4, Shengzhen A
    df=pd.read_csv('csmarstk0016.csv')
    df['TRDMNT'] = pd.to_datetime(df['TRDMNT'].astype('str'), format='%Y-%m-%d')
    df = df.rename(columns={'TRDMNT':'date'})
    df = df.loc[df['ret'].notnull()]
    return df

In [ ]:
stkret = load_stock_returns()
stkret.head()

In [ ]:
def load_market_return():
    df = pd.read_csv('csmarmktretrf.csv')
    df['TRDMNT'] = pd.to_datetime(df['TRDMNT'].astype('str'), format='%Y-%m-%d')
    df = df.rename(columns={'TRDMNT':'date'})
    df = df.drop('mktretxd', axis=1)
    return df

In [ ]:
mktret = load_market_return()
mktret.head(3)

In [ ]:
def load_book_value():
    # December Book Equity where BE >0
    df= pd.read_csv('csmar_be.csv')
    df['date'] = pd.to_datetime(df['date'].astype('str'), format='%Y-%m-%d') 
    df = df.loc[df['BE']>0]
    df = df.sort_values(["STKCD", 'date', 'BE'])
    df = df.groupby(['STKCD', 'date']).last().reset_index()
    return df

In [ ]:
decbe = load_book_value()
decbe.head()

### Q2.1 
Assume that power utility and relative risk aversion is 3, and constent expected market excess return.
* What is the optimal proportion of wealth to invest in the risky (stock) assets? (you will invest in a market portfolio). 
* Your answer above may be different from the typical answer in the U.S. market, explain what mainly drives this difference?

### Size and Value

#### Forming Portfolios

According to size and MTB.

Starting from '2000-7-31', form a 5X5 portfolios of size and book-to-market. Size is defined by end of June market value of equity, value is defined by December book value of equity in the previous year. Portfolio is rebalanced at each July. 
* You may want to create a 'year' variable that designate portfolio formation year.

In [ ]:
# year variable here is the portfolio formation year
stkret['year'] = np.where(stkret['date'].dt.month>=7, stkret['date'].dt.year, stkret['date'].dt.year-1)

### Q2.2 
Show the CAPM alpha and beta of these portfolios in respective tables

### Q2.3
Calculate actual mean portfolio returns and compare them with the CAPM predicted average portfolio returns, explain any patterns in the data. 

### Q2.4 
Plot the mean excess return against CAPM predicted return, plot both within size (BM) plots and the within BM (size) plots. In the U.S. data, the most severe violation of CAPM happens along BTM portfolios, What about China? Can you explain why this factor comes up so significant in the Chinese markets?

### Constructing SMB and HML factors

Construct 2x3 portfolios of size and BTM according to the Shanghai Stock Exchange cutoff (MARKETTYPE==1). BTM cutoff is (0.3,0.7), Size cutoff is 0.5. Calculate the value-weighted returns of the portfolios and then construct SMB and HML factors following Fama French (93) procedure. 

### Q2.5
Plot the cumulative returns of the SMB and HML factors for the Chinese marlet. 

### Q2.6 
Tabulate the coefficient estimates of the Fama-French 3 factor model of the 25 (5x5) sorted portfolio returns. Does the coefficients estimates make sense? 

### Q2.7 
plot the Mean portfolio return against the Fama-French 3 factor predicted return again for both Within-BM Size Plot and Within-Size BM Plot. How do the graphs compare to the earlier CAPM plots? 